In [1]:
%matplotlib inline

In [2]:
import model_utils
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [3]:
data_dir = 'data/masked_images/data/trainval/breeds_dog/'
test_data_dir = 'data/masked_images/data/test/breeds_dog/'
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
train_transforms = transforms.Compose([transforms.Resize(224),
                                        transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       normalize
                                       ])
test_transforms = transforms.Compose([transforms.Resize(224),
                                        transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       normalize
                                       ])
trainloader, valloader, classes = model_utils.load_data('breeds_dog', train_transforms, test_transforms, 1)
#trainloader, valloader, classes, labels = model_utils.load_split_train_val(data_dir, train_transforms,test_transforms, normalize,.25)

In [4]:
use_gpu = torch.cuda.is_available()

model_dog = torchvision.models.resnet50(pretrained=True)
num_ftrs = model_dog.fc.in_features

model_dog.fc = nn.Linear(num_ftrs, len(classes))

if use_gpu:
    model_dog = model_dog.cuda()

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_dog.parameters(), lr=0.001)

scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

In [ ]:
model_dog = model_utils.train_model(model=model_dog,
                           criterion=criterion,
                           optimizer=optimizer_ft,
                           scheduler=scheduler,
                           trainloader=trainloader,
                           valloader=valloader,
                           num_epochs=10)

Epoch 0/9
--------------------
train Loss: 3.3087 Acc: 0.0404
val Loss: 3.0179 Acc: 0.0886
--------------------
Epoch 1/9
--------------------
train Loss: 3.2211 Acc: 0.0576
val Loss: 2.5487 Acc: 0.3808
--------------------
Epoch 2/9
--------------------
train Loss: 2.8779 Acc: 0.1805
val Loss: 1.7730 Acc: 0.5668
--------------------
Epoch 3/9
--------------------
train Loss: 1.8494 Acc: 0.5282
val Loss: 1.2836 Acc: 0.6330
--------------------
Epoch 4/9
--------------------
train Loss: 1.0547 Acc: 0.7491
val Loss: 1.0654 Acc: 0.7640
--------------------
Epoch 5/9
--------------------
train Loss: 0.6052 Acc: 0.9085
val Loss: 1.1169 Acc: 0.7765
--------------------
Epoch 6/9
--------------------
train Loss: 0.5243 Acc: 0.9345
val Loss: 1.1637 Acc: 0.7591
--------------------
Epoch 7/9
--------------------
train Loss: 0.4743 Acc: 0.9424
val Loss: 1.2742 Acc: 0.7416
--------------------
Epoch 8/9
--------------------
train Loss: 0.4327 Acc: 0.9539


In [ ]:
saving_path = './trained_model/resnet50_breed_dog.pth'
torch.save(model_dog, saving_path)

In [ ]:
model_utils.print_test_acc(model_dog, test_transforms, normalize, test_data_dir)